In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

In [2]:
df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

lst1 = list()
for col in X:
    if len(X[col].unique()) > 2:
        lst1.append(col)

scaler = StandardScaler()
X[lst1] = scaler.fit_transform(X[lst1])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
models = {"svm": {"model": SVC(),
                    "params": {
                            "C": range(1, 11),
                            "kernel": ["linear", "poly", "rbf",],
                            "gamma": ["scale", "auto"]
                            }
                    },
          "tree":{"model": DecisionTreeClassifier(),
                  "params": {
                      "criterion": ["gini", "entropy"],
                      "splitter": ["best", "random"],
                      "max_depth": [None, 1, 3, 5],
                            },
                    },
          "nb":{"model": GaussianNB(),
                "params": {},
                    },
          "rf":{"model": RandomForestClassifier(),
                "params": {
                    "n_estimators": [1, 3, 5, 10, 20, 50, 100],
                    "criterion": ["gini", "entropy"],
                    "max_depth": [None, 1, 3, 5],
                    },
                }
          }

scores = []
for m, p in models.items():
    clf = GridSearchCV(p["model"], p["params"], cv=5)
    clf.fit(X, y)
    scores.append({"model" : m,
                   "best_score" : clf.best_score_,
                   "best_params" : clf.best_params_})

df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])

In [6]:
df


,model,best_score,best_params
0,svm,0.825876,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}"
1,tree,0.839548,"{'criterion': 'gini', 'max_depth': 1, 'splitte..."
2,nb,0.759040,{}
3,rf,0.852881,"{'criterion': 'entropy', 'max_depth': 3, 'n_es..."
